In [117]:
using BenchmarkTools
using OSMToolset
using PyCall
using Colors
using DataFrames
using Plots
using OpenStreetMapX, OpenStreetMapXPlot

In [53]:
f2 = download("https://overpass-api.de/api/map?bbox=-71.0930,42.3397,-71.0082,42.3875")
mv(f2, "BostonToTests.osm",force=true)

"BostonToTests.osm"

In [54]:
map_data = get_map_data("BostonToTests.osm"; use_cache=false, only_intersections=false,trim_to_connected_graph=true);

In [95]:
using Random
#Random.seed!(0);
a,b = [point_to_nodes(generate_point_in_bounds(map_data), map_data) for _ in 1:2]
    route, route_time = OpenStreetMapX.shortest_route(map_data,a,b)

node_ids = collect(keys(map_data.nodes)) 
visits = Dict{Int,Int}()
for n in route
    visits[n] = get(visits, n,0)+1
end 

In [118]:
@btime OpenStreetMapX.shortest_route(map_data,a,b)

  19.573 ms (280 allocations: 7.26 MiB)


([61362466, 5456441901, 1108415965, 61362486, 1103944294, 1108415961, 7852121709, 61345675, 7801832061, 61345676  …  6346137676, 61425923, 61608915, 61608869, 1001627975, 61608875, 6346137675, 6346137674, 6346137673, 6346137672], 9431.736474630066, 827.2697776581225)

In [119]:
@btime OpenStreetMapX.shortest_route(map_data,a,b, routing = :dijkstra)

  21.348 ms (470 allocations: 8.25 MiB)


([61362466, 5456441901, 1108415965, 61362486, 1103944294, 1108415961, 7852121709, 61345675, 7801832061, 61345676  …  6346137676, 61425923, 61608915, 61608869, 1001627975, 61608875, 6346137675, 6346137674, 6346137673, 6346137672], 9431.736474630066, 827.2697776581225)

In [97]:
flm = pyimport("folium")
matplotlib_cm = pyimport("matplotlib.cm")
matplotlib_colors = pyimport("matplotlib.colors")
cmap = matplotlib_cm.get_cmap("prism")

SHOW_PATHS=1
m = flm.Map()
locs = [LLA(map_data.nodes[n],map_data.bounds) for n in route]

flm.PolyLine(        
    [(loc.lat, loc.lon) for loc in locs ],
).add_to(m)

MAP_BOUNDS = [(map_data.bounds.min_y,map_data.bounds.min_x),(map_data.bounds.max_y,map_data.bounds.max_x)]
flm.Rectangle(MAP_BOUNDS, color="black",weight=6).add_to(m)
m.fit_bounds(MAP_BOUNDS)
m

PyObject <folium.folium.Map object at 0x000000005B4ED330>